In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc
from collections import Counter
#import cudf, 
import itertools
#print('We will use RAPIDS version',cudf.__version__)
import warnings
warnings.filterwarnings('ignore')

# load

In [4]:
INPUT_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/input/'
INPUT_DIR_2 = '/content/drive/MyDrive/kaggle/2022/OTTO/output/candidate/'
OUTPUT_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/output/candidate/'

In [5]:
#ここはchris baselineと同じ。
#https://www.kaggle.com/code/cdeotte/compute-validation-score-cv-565

type_labels = {'clicks':0, 'carts':1, 'orders':2}

def load_test():    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(INPUT_DIR+'cris_baseline/otto-validation/test_parquet/*')):
        chunk = pd.read_parquet(chunk_file)
        #chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

#test_df
test_df = load_test()
print('Test data has shape',test_df.shape)
test_df

Test data has shape (7683577, 4)


,session,aid,ts,type
0,11188591,1485015,1661165822000,0
1,11188591,1562739,1661165866000,0
2,11188591,1485015,1661165882000,0
3,11188591,1441634,1661165891000,0
4,11188591,1485015,1661165901000,0
...,...,...,...,...
7683572,12629597,744187,1661646518000,0
7683573,12629597,714524,1661646521000,0
7683574,12629597,1844379,1661646612000,0
7683575,12629598,894387,1661646273000,0


In [6]:
#自作のtest dayごとの人気商品top20 読み込み

test_day = [21, 22, 23, 24, 25, 26, 27, 28]
freq_aids = {}

for day in test_day:
  with open(INPUT_DIR_2 + f'test_popular_day{day}.pkl','rb') as f:
    freq_aid = pickle.load(f)
    freq_aids.update(freq_aid)

In [7]:
freq_aids

{21: [108125,
  1603001,
  29735,
  1768724,
  558573,
  984459,
  166037,
  1539309,
  1022566,
  1733943,
  225619,
  1122221,
  1682750,
  1505419,
  615771,
  544144,
  224347,
  831165,
  1426624,
  95488],
 22: [1460571,
  166037,
  579690,
  1733943,
  1596897,
  331708,
  108125,
  554660,
  29735,
  195381,
  332654,
  1604220,
  1022566,
  836852,
  184976,
  634452,
  620545,
  530377,
  1552037,
  986164],
 23: [485256,
  33343,
  152547,
  1551213,
  1765072,
  1736857,
  660655,
  1562705,
  803544,
  1460571,
  321547,
  331708,
  579690,
  1236775,
  184976,
  1043508,
  836852,
  108125,
  1733943,
  1604220],
 24: [613493,
  1406660,
  485256,
  1551213,
  1460571,
  1236775,
  108125,
  1531805,
  184976,
  19665,
  959208,
  836852,
  1294924,
  1347107,
  738987,
  554660,
  231487,
  166037,
  29735,
  331708],
 25: [876493,
  1460571,
  485256,
  108125,
  1551213,
  1236775,
  1531805,
  184976,
  836852,
  1406660,
  331708,
  1116095,
  554660,
  231487,
  634

# inference

In [8]:
#65時間かかる
#0% 1155/1801251 [02:31<65:51:14, 7.59it/s]

'''
test_df['date'] = pd.to_datetime(test_df['ts'], unit='ms')
test_df['day'] = test_df['date'].dt.strftime('%d')
unique_session = test_df['session'].unique()
test_df['labels'] = 1

#セッションごとにdayの人気商品を予測にする
for session in tqdm(unique_session):
  day = int(test_df.loc[test_df['session']==session,'day'].max())
  test_df.loc[test_df['session']==session, 'labels'] = ' '.join(map(str, freq_aids[day]))
'''

"\ntest_df['date'] = pd.to_datetime(test_df['ts'], unit='ms')\ntest_df['day'] = test_df['date'].dt.strftime('%d')\nunique_session = test_df['session'].unique()\ntest_df['labels'] = 1\n\n#セッションごとにdayの人気商品を予測にする\nfor session in tqdm(unique_session):\n  day = int(test_df.loc[test_df['session']==session,'day'].max())\n  test_df.loc[test_df['session']==session, 'labels'] = ' '.join(map(str, freq_aids[day]))\n"

In [9]:
#85時間
#0% 1640/1801251 [04:43<85:41:08, 5.83it/s]

'''
test_df['date'] = pd.to_datetime(test_df['ts'], unit='ms')
test_df['day'] = test_df['date'].dt.strftime('%d')
unique_session = test_df['session'].unique()
test_df['labels'] = 1

#セッションごとにdayの人気商品を予測にする

for session in tqdm(unique_session):
  for day in test_day:
    if day == int(test_df.loc[test_df['session']==session,'day'].max()):
      test_df.loc[test_df['session']==session, 'labels'] = ' '.join(map(str, freq_aids[day]))
'''

"\ntest_df['date'] = pd.to_datetime(test_df['ts'], unit='ms')\ntest_df['day'] = test_df['date'].dt.strftime('%d')\nunique_session = test_df['session'].unique()\ntest_df['labels'] = 1\n\n#セッションごとにdayの人気商品を予測にする\n\nfor session in tqdm(unique_session):\n  for day in test_day:\n    if day == int(test_df.loc[test_df['session']==session,'day'].max()):\n      test_df.loc[test_df['session']==session, 'labels'] = ' '.join(map(str, freq_aids[day]))\n"

In [10]:
#実行時間早くするために、処理済みのsessionはtest_dfから削除して徐々にtest_df減らしていく。
#をやったら大幅に悪化して600時間かかる。tqdmの仕様わからんけどそれでも後から時間短縮されてくだろうけど厳しそう。
#0% 869/1801251 [18:18<625:08:32, 1.25s/it]
'''
test_df['date'] = pd.to_datetime(test_df['ts'], unit='ms')
test_df['day'] = test_df['date'].dt.strftime('%d')
unique_session = test_df['session'].unique()
test_df['labels'] = 1

#セッションごとにdayの人気商品を予測にする
tmp_test_df = test_df.copy()
for session in tqdm(unique_session):
  day = int(tmp_test_df.loc[tmp_test_df['session']==session,'day'].max())
  test_df.loc[test_df['session']==session, 'labels'] = ' '.join(map(str, freq_aids[day]))
  tmp_test_df = tmp_test_df.drop(index=list(tmp_test_df.loc[tmp_test_df['session']==session].index))
'''

"\ntest_df['date'] = pd.to_datetime(test_df['ts'], unit='ms')\ntest_df['day'] = test_df['date'].dt.strftime('%d')\nunique_session = test_df['session'].unique()\ntest_df['labels'] = 1\n\n#セッションごとにdayの人気商品を予測にする\ntmp_test_df = test_df.copy()\nfor session in tqdm(unique_session):\n  day = int(tmp_test_df.loc[tmp_test_df['session']==session,'day'].max())\n  test_df.loc[test_df['session']==session, 'labels'] = ' '.join(map(str, freq_aids[day]))\n  tmp_test_df = tmp_test_df.drop(index=list(tmp_test_df.loc[tmp_test_df['session']==session].index))\n"

In [12]:
#荒技 dayごとのnotebook分かるする場合のcode.
#day1あたり3時間くらい。やりたくない
'''
target_day = 21

test_df['date'] = pd.to_datetime(test_df['ts'], unit='ms')
test_df['day'] = test_df['date'].dt.strftime('%d')
unique_session = test_df['session'].unique()
test_df['labels'] = 1

#セッションごとにdayの人気商品を予測にする
for session in tqdm(unique_session):
  if target_day == int(test_df.loc[test_df['session']==session,'day'].max()):
    test_df.loc[test_df['session']==session, 'labels'] = ' '.join(map(str, freq_aids[target_day]))
'''

"\ntarget_day = 21\n\ntest_df['date'] = pd.to_datetime(test_df['ts'], unit='ms')\ntest_df['day'] = test_df['date'].dt.strftime('%d')\nunique_session = test_df['session'].unique()\ntest_df['labels'] = 1\n\n#セッションごとにdayの人気商品を予測にする\nfor session in tqdm(unique_session):\n  if target_day == int(test_df.loc[test_df['session']==session,'day'].max()):\n    test_df.loc[test_df['session']==session, 'labels'] = ' '.join(map(str, freq_aids[target_day]))\n"

# save

In [13]:
#test_df.to_csv(OUTPUT_DIR + f'test_popular_day{target_day}.csv',index = False)

NameError: ignored

# load

In [ ]:
#test = pd.read_csv(OUTPUT_DIR + f'test_popular_day{target_day}.csv')
#test